In [1]:
import requests
import time
import csv
from tqdm import tqdm
import pandas as pd

In [2]:
rawg_api = 'https://api.rawg.io/api/games' # ?page=<number>&page_size=<number>
def list_games(json):
    games = []
    for result in json['results']:
        genres = get_genres(result)
        games.append([result['id'], result['name'], result['rating'], 
                      result['ratings_count'], result['ratings'], genres, 
                      result['released']])
    return games

In [3]:
def get_genres(result):
    return [genre['name'] for genre in result['genres']]

In [4]:
def to_csv(game, filename):
    with open(f'{filename}.csv', 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(game)

In [5]:
def call_api(api, pages, filename):
    not_added =[]
    total_games = []
    for page in tqdm(range(pages[0], pages[1])):
        json = requests.get(f'{api}?page={page}&page_size=40').json()
        games = list_games(json)
        try:
            for game in games:
                to_csv(game, filename)
        except:
            not_added.append(game)
        time.sleep(1)       
    return not_added

In [6]:
not_added = call_api(rawg_api, [1, 8260], 'Rawg_genre_data')

100%|████████████████████████████████████████████████████████████████████████████| 8259/8259 [4:31:27<00:00,  1.97s/it]


In [7]:
df = pd.read_csv('Rawg_genre_data.csv', encoding='ISO-8859-1',
                names=['Rawg_id', 'Game', 'Rawg_rating', 'Rawg_vote_count',
                        'Rating_Bdown', 'Genres', 'Released'])

In [46]:
new_df = df[df['Rawg_vote_count']>=1]
new_df.to_csv('votes_only_raw_data.csv', index=False)

In [ ]:
import re

df2 = df['Genre'].str.replace(r']', '')
df2 = pd.DataFrame(df2.str.split().values.tolist())
df2[0] = df2[0].str.replace(r'[', '')
df2